In [1]:
import pandas as pd
import numpy as np

In [2]:
# 종로구 내의 교통사고다발지역 데이터 불러오기
jongro = pd.read_excel('./data/교통사고_종로구.xlsx')
jongro

FileNotFoundError: [Errno 2] No such file or directory: './data/교통사고_종로구.xlsx'

In [4]:
# 반경 내 데이터 필터링을 위한 클래스 생성
from geopy.distance import great_circle
import folium

class CountByWGS84:

    def __init__(self, df, lat, lon, dist=1):
        """
        df: 데이터 프레임
        lat: 중심 위도
        lon: 중심 경도
        dist: 기준 거리(km)
        """
        self.df = df
        self.lat = lat
        self.lon = lon
        self.dist = dist

    def filter_by_rectangle(self):
        """
        사각 범위 내 데이터 필터링
        """
        lat_min = self.lat - 0.01 * self.dist
        lat_max = self.lat + 0.01 * self.dist

        lon_min = self.lon - 0.015 * self.dist
        lon_max = self.lon + 0.015 * self.dist

        self.points = [[lat_min, lon_min], [lat_max, lon_max]]

        result = self.df.loc[
            (self.df['lat'] > lat_min) &
            (self.df['lat'] < lat_max) &
            (self.df['lon'] > lon_min) &
            (self.df['lon'] < lon_max)
        ]
        result.index = range(len(result))

        return result

    def filter_by_radius(self):
        """
        반경 범위 내 데이터 필터링
        """
        # 사각 범위 내 데이터 필터링
        tmp = self.filter_by_rectangle()

        # 기준 좌표 포인트
        center = (self.lat, self.lon)

        result = pd.DataFrame()
        for index, row in tmp.iterrows():
            # 개별 좌표 포인트
            point = (row['lat'], row['lon'])
            d = great_circle(center, point).kilometers
            if d <= self.dist:
                result = pd.concat([result, tmp.iloc[index, :].to_frame().T])

        result.index = range(len(result))

        return result

    def plot_by_rectangle(self, df):
        """
        사각 범위 내 데이터 플로팅
        """

        m = folium.Map(location=[self.lat, self.lon], zoom_start=14)

        for idx, row in df.iterrows():

            lat_ = row['lat']
            lon_ = row['lon']

            folium.Marker(location=[lat_, lon_],
                          radius=15,
                          tooltip=row['도로노선명']).add_to(m)
            folium.Marker([self.lat, self.lon], icon=(folium.Icon(color="red"))).add_to(m)

        folium.Rectangle(bounds=self.points,
                         color='#ff7800',
                         fill=True,
                         fill_color='#ffff00',
                         fill_opacity=0.2).add_to(m)

        return m

    def plot_by_radius(self, df):
        """
        반경 범위 내 데이터 플로팅
        """

        m = folium.Map(location=[self.lat, self.lon], zoom_start=14)

        for idx, row in df.iterrows():

            lat_ = row['lat']
            lon_ = row['lon']

            folium.Marker(location=[lat_, lon_],
                          radius=15,
                          tooltip=row['도로노선명']).add_to(m)
            folium.Marker([self.lat, self.lon], icon=(folium.Icon(color="red"))).add_to(m)

        folium.Circle(radius=dist * 1000,
                      location=[lat, lon],
                      color="#ff7800",
                      fill_color='#ffff00',
                      fill_opacity=0.2
                      ).add_to(m)

        return m

#### 1. cctv 갯수 카운트

In [5]:
# 서울시 내 cctv 데이터 불러오기
cctv_df = pd.read_excel('./data/cctv데이터.xlsx')
cctv_df = cctv_df.rename(columns={'위도':'lat', '경도':'lon'})
cctv_df

,시도명,시군구명,도로노선명,소재지도로명주소,lat,lon
0,서울특별시,강동구,진황도로,서울특별시 강동구 진황도로 159,37.531926,127.141845
1,서울특별시,강동구,구천면로,서울특별시 강동구 구천면로 196,37.540453,127.126322
2,서울특별시,강동구,상암로,서울특별시 강동구 상암로 200,37.546143,127.144533
3,서울특별시,강동구,성내로,서울특별시 강동구 성내로 22,37.529561,127.122801
4,서울특별시,강동구,고덕로,서울특별시 강동구 고덕로 262,37.554668,127.154620
...,...,...,...,...,...,...
1560,서울특별시,광진구,강변역로,서울특별시 광진구 강변역로 53,37.535336,127.094313
1561,서울특별시,광진구,광나루로56길,서울특별시 광진구 광나루로56길 32,37.537871,127.096219
1562,서울특별시,광진구,능동로,서울특별시 광진구 어린이대공원역1번출구,37.548591,127.075542
1563,서울특별시,광진구,능동로,서울특별시 광진구 능동로 248,37.552162,127.076962


In [6]:
# # 파라미터 설정 => 반경거리, 반경의 중심점 설정
# lat = 37.585533
# lon = 127.001217
# dist = 1

In [7]:
# # 반경 집계 인스턴스 생성
# cbw = CountByWGS84(cctv_df, lat, lon, dist)

In [8]:
# # 반경 범위 내 데이터 필터링
# result = cbw.filter_by_radius()
# cbw.plot_by_radius(result)

In [9]:
# # 반경 범위 내 데이터 필터링 결과
# len(result)

In [10]:
cctv_count = []

for idx, val in jongro.iterrows():
    # 파라미터 설정 => 반경거리, 반경의 중심점 설정
    lat = val['위도']
    lon = val['경도']
    dist = 1
    
    # 반경 집계 인스턴스 생성
    cbw = CountByWGS84(cctv_df, lat, lon, dist)
    
    # 반경 범위 내 데이터 필터링
    result = cbw.filter_by_radius()
    
    # 반경 범위 내 데이터 필터링 결과 리스트에 추가
    cctv_count.append(len(result))

In [11]:
len(cctv_count)

843

In [15]:
# cctv_count 리스트를 데이터프레임으로 변경
cctv_df = pd.DataFrame(cctv_count)
cctv_df.columns = ['cctv 갯수']

In [53]:
jongro_df = pd.concat([jongro, cctv_df], axis=1)
jongro_df

,Column1,년도,월,일,요일,시각,사고등급,사망자수,중상자수,경상자수,...,일광상태,사고유형1,사고유형2,법규위반,도로종류,지점명,위도,경도,자치구,cctv 갯수
0,0,2017,7,13,목요일,9,부상사고,0,0,0,...,주간,차대차,측면직각충돌,안전거리미확보,특별광역시도,관철동14-1번지사거리,37.570176,126.983197,종로구,33
1,1,2017,9,1,금요일,3,중상사고,0,1,0,...,심야,차대차,차대차-기타,안전운전불이행,특별광역시도,관철동14-1번지사거리,37.570176,126.983197,종로구,33
2,2,2017,11,24,금요일,1,경상사고,0,0,1,...,심야,차대사람,횡단중,신호위반,특별광역시도,관철동14-1번지사거리,37.570176,126.983197,종로구,33
3,3,2018,3,24,토요일,20,경상사고,0,0,1,...,출퇴근,차대사람,차도통행중,안전운전불이행,특별광역시도,관철동14-1번지사거리,37.570176,126.983197,종로구,33
4,4,2018,7,28,토요일,18,경상사고,0,0,2,...,출퇴근,차대차,차대차-기타,안전거리미확보,특별광역시도,관철동14-1번지사거리,37.570176,126.983197,종로구,33
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
838,76,2019,11,15,금요일,7,경상사고,0,0,1,...,출퇴근,차대차,차대차-기타,안전거리미확보,특별광역시도,혜화동로터리,37.585533,127.001217,종로구,32
839,77,2019,11,7,목요일,8,경상사고,0,0,2,...,출퇴근,차대차,측면직각충돌,안전운전불이행,특별광역시도,혜화동로터리,37.585533,127.001217,종로구,32
840,78,2019,11,15,금요일,8,경상사고,0,0,1,...,출퇴근,차대차,측면직각충돌,안전거리미확보,특별광역시도,혜화동로터리,37.585533,127.001217,종로구,32
841,79,2019,12,10,화요일,6,경상사고,0,0,1,...,새벽,차대차,측면직각충돌,신호위반,특별광역시도,혜화동로터리,37.585533,127.001217,종로구,32


#### 2. 표지판 갯수 카운트

In [24]:
# 서울시 내 표지판 데이터 불러오기
pyogi = pd.read_csv('./data/서울특별시 도로표지 설치현황.csv', encoding='cp949')
pyogi = pyogi[['표지일련번호', '표지종별', 'X좌표', 'Y좌표', '데이터기준일자']]
pyogi

,표지일련번호,표지종별,X좌표,Y좌표,데이터기준일자
0,WR(서울특별시 양천구)-[목동로3길]-하-7,3방향표지,188389.16,445733.57,2018-04-30
1,WR(서울특별시 도봉구)-[도봉로]-상-7,3방향표지,203697.47,462397.48,2018-04-30
2,WR(서울특별시 양천구)-[목동동로8길]-하-1,2방향예고표지,188569.00,446003.28,2018-04-30
3,WR(서울특별시 양천구)-[목동동로6길]-상-1,3방향예고표지,188144.06,445857.85,2018-04-30
4,UR(서울특별시)-[도봉로]-상-7,3방향예고표지,203602.74,461995.14,2018-04-30
...,...,...,...,...,...
10936,NR-48[개화동로]-상-33,2방향예고표지,182129.47,453098.23,2018-04-30
10937,NR-48[개화동로]-하-142,1방향표지,182653.76,452066.11,2018-04-30
10938,NR-48[개화동로]-하-143,2방향표지,182653.76,452066.11,2018-04-30
10939,WR(서울특별시 강서구)-48[초원로]-하-1,2방향표지,182769.00,451761.00,2018-04-30


In [20]:
# 중부TM 좌표계 => WGS84좌표계 (일반적인 위, 경도) 변환 필요
from pyproj import Proj, transform

# Projection 정의
# 중부원점(Bessel): 서울 등 중부지역 EPSG:2097
proj_1 = Proj(init='epsg:2097')

# WGS84 경위도: GPS가 사용하는 좌표계 EPSG:4326
proj_2 = Proj(init='epsg:4326')

D:\Anaconda\lib\site-packages\pyproj\crs\crs.py:68: FutureWarning: '+init=<authority>:<code>' syntax is deprecated. '<authority>:<code>' is the preferred initialization method. When making the change, be mindful of axis order changes: https://pyproj4.github.io/pyproj/stable/gotchas.html#axis-order-changes-in-proj-6
  return _prepare_from_string(" ".join(pjargs))
D:\Anaconda\lib\site-packages\pyproj\crs\crs.py:306: FutureWarning: '+init=<authority>:<code>' syntax is deprecated. '<authority>:<code>' is the preferred initialization method. When making the change, be mindful of axis order changes: https://pyproj4.github.io/pyproj/stable/gotchas.html#axis-order-changes-in-proj-6
  projstring = _prepare_from_string(" ".join((projstring, projkwargs)))
D:\Anaconda\lib\site-packages\pyproj\crs\crs.py:68: FutureWarning: '+init=<authority>:<code>' syntax is deprecated. '<authority>:<code>' is the preferred initialization method. When making the change, be mindful of axis order changes: https://py

In [35]:
# 데이터 프레임 지정
pyogi_copy = pyogi.copy()

x_list = []
y_list = []

for idx, row in pyogi_copy.iterrows():
    x, y = row['X좌표'], row['Y좌표']
    x_, y_ = transform(proj_1, proj_2, x, y)
    x_list.append(x_)
    y_list.append(y_)
    
pyogi['lon'] = x_list
pyogi['lat'] = y_list

<ipython-input-35-3befa6ea0d71>:9: DeprecationWarning: This function is deprecated. See: https://pyproj4.github.io/pyproj/stable/gotchas.html#upgrading-to-pyproj-2-from-pyproj-1
  x_, y_ = transform(proj_1, proj_2, x, y)


In [36]:
pyogi

,표지일련번호,표지종별,X좌표,Y좌표,데이터기준일자,lon,lat
0,WR(서울특별시 양천구)-[목동로3길]-하-7,3방향표지,188389.16,445733.57,2018-04-30,126.866569,37.513754
1,WR(서울특별시 도봉구)-[도봉로]-상-7,3방향표지,203697.47,462397.48,2018-04-30,127.039803,37.663960
2,WR(서울특별시 양천구)-[목동동로8길]-하-1,2방향예고표지,188569.00,446003.28,2018-04-30,126.868599,37.516186
3,WR(서울특별시 양천구)-[목동동로6길]-상-1,3방향예고표지,188144.06,445857.85,2018-04-30,126.863795,37.514871
4,UR(서울특별시)-[도봉로]-상-7,3방향예고표지,203602.74,461995.14,2018-04-30,127.038727,37.660335
...,...,...,...,...,...,...,...
10936,NR-48[개화동로]-상-33,2방향예고표지,182129.47,453098.23,2018-04-30,126.795586,37.580008
10937,NR-48[개화동로]-하-142,1방향표지,182653.76,452066.11,2018-04-30,126.801546,37.570719
10938,NR-48[개화동로]-하-143,2방향표지,182653.76,452066.11,2018-04-30,126.801546,37.570719
10939,WR(서울특별시 강서구)-48[초원로]-하-1,2방향표지,182769.00,451761.00,2018-04-30,126.802858,37.567972


In [37]:
pyogi_count = []

for idx, val in jongro.iterrows():
    lat = val['위도']
    lon = val['경도']
    dist = 1
    
    # 반경 집계 인스턴스 생성
    cbw = CountByWGS84(pyogi, lat, lon, dist)
    
    # 반경 범위 내 데이터 필터링
    result = cbw.filter_by_radius()
    
    # 반경 범위 내 데이터 필터링 결과 리스트에 추가
    pyogi_count.append(len(result))

In [38]:
len(pyogi_count)

843

In [39]:
# pyogi_count 리스트를 데이터프레임으로 변경
pyogi_df = pd.DataFrame(pyogi_count)
pyogi_df.columns = ['표지판 갯수']

In [54]:
jongro_df = pd.concat([jongro_df, pyogi_df], axis=1)

,Column1,년도,월,일,요일,시각,사고등급,사망자수,중상자수,경상자수,...,사고유형1,사고유형2,법규위반,도로종류,지점명,위도,경도,자치구,cctv 갯수,표지판 갯수
0,0,2017,7,13,목요일,9,부상사고,0,0,0,...,차대차,측면직각충돌,안전거리미확보,특별광역시도,관철동14-1번지사거리,37.570176,126.983197,종로구,33,198
1,1,2017,9,1,금요일,3,중상사고,0,1,0,...,차대차,차대차-기타,안전운전불이행,특별광역시도,관철동14-1번지사거리,37.570176,126.983197,종로구,33,198
2,2,2017,11,24,금요일,1,경상사고,0,0,1,...,차대사람,횡단중,신호위반,특별광역시도,관철동14-1번지사거리,37.570176,126.983197,종로구,33,198
3,3,2018,3,24,토요일,20,경상사고,0,0,1,...,차대사람,차도통행중,안전운전불이행,특별광역시도,관철동14-1번지사거리,37.570176,126.983197,종로구,33,198
4,4,2018,7,28,토요일,18,경상사고,0,0,2,...,차대차,차대차-기타,안전거리미확보,특별광역시도,관철동14-1번지사거리,37.570176,126.983197,종로구,33,198
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
838,76,2019,11,15,금요일,7,경상사고,0,0,1,...,차대차,차대차-기타,안전거리미확보,특별광역시도,혜화동로터리,37.585533,127.001217,종로구,32,32
839,77,2019,11,7,목요일,8,경상사고,0,0,2,...,차대차,측면직각충돌,안전운전불이행,특별광역시도,혜화동로터리,37.585533,127.001217,종로구,32,32
840,78,2019,11,15,금요일,8,경상사고,0,0,1,...,차대차,측면직각충돌,안전거리미확보,특별광역시도,혜화동로터리,37.585533,127.001217,종로구,32,32
841,79,2019,12,10,화요일,6,경상사고,0,0,1,...,차대차,측면직각충돌,신호위반,특별광역시도,혜화동로터리,37.585533,127.001217,종로구,32,32


In [58]:
jongro_df = jongro_df.drop(['Column1'],axis=1)
jongro_df

,년도,월,일,요일,시각,사고등급,사망자수,중상자수,경상자수,부상자수,...,사고유형1,사고유형2,법규위반,도로종류,지점명,위도,경도,자치구,cctv 갯수,표지판 갯수
0,2017,7,13,목요일,9,부상사고,0,0,0,1,...,차대차,측면직각충돌,안전거리미확보,특별광역시도,관철동14-1번지사거리,37.570176,126.983197,종로구,33,198
1,2017,9,1,금요일,3,중상사고,0,1,0,0,...,차대차,차대차-기타,안전운전불이행,특별광역시도,관철동14-1번지사거리,37.570176,126.983197,종로구,33,198
2,2017,11,24,금요일,1,경상사고,0,0,1,0,...,차대사람,횡단중,신호위반,특별광역시도,관철동14-1번지사거리,37.570176,126.983197,종로구,33,198
3,2018,3,24,토요일,20,경상사고,0,0,1,1,...,차대사람,차도통행중,안전운전불이행,특별광역시도,관철동14-1번지사거리,37.570176,126.983197,종로구,33,198
4,2018,7,28,토요일,18,경상사고,0,0,2,0,...,차대차,차대차-기타,안전거리미확보,특별광역시도,관철동14-1번지사거리,37.570176,126.983197,종로구,33,198
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
838,2019,11,15,금요일,7,경상사고,0,0,1,0,...,차대차,차대차-기타,안전거리미확보,특별광역시도,혜화동로터리,37.585533,127.001217,종로구,32,32
839,2019,11,7,목요일,8,경상사고,0,0,2,0,...,차대차,측면직각충돌,안전운전불이행,특별광역시도,혜화동로터리,37.585533,127.001217,종로구,32,32
840,2019,11,15,금요일,8,경상사고,0,0,1,0,...,차대차,측면직각충돌,안전거리미확보,특별광역시도,혜화동로터리,37.585533,127.001217,종로구,32,32
841,2019,12,10,화요일,6,경상사고,0,0,1,0,...,차대차,측면직각충돌,신호위반,특별광역시도,혜화동로터리,37.585533,127.001217,종로구,32,32


In [60]:
jongro_df.to_csv('./교통사고데이터(종로구)+cctv,표지판.csv', encoding='utf-8-sig', index=None)